This kernel had only 0.5788 score in Public Leaderboard (~1090 place).
Nevertheless, I choosed it for the final submission, as I believed it can generalize well.

But the result was even better then I expected! The kernel endet up at place 20 with score 0.7536.

Well, I think it's a good example why we should not always trust Public Leaderboard :)


Many thanks to Nanashi and to Markus F, whose kernels ([#1 Smart Robots. Most Complete Notebook ](http://www.kaggle.com/jesucristo/1-smart-robots-most-complete-notebook) and [The Missing Link...](http://www.kaggle.com/friedchips/the-missing-link)) helped a lot.

**Description of the task from the competion page**
> 
> Robots are smart… by design. To fully understand and properly navigate a task, however, they need input about their environment.
> 
> In this competition, you’ll help robots recognize the floor surface they’re standing on using data collected from Inertial Measurement Units (IMU sensors).
> 
> We’ve collected IMU sensor data while driving a small mobile robot over different floor surfaces on the university premises. The task is to predict which one of the nine floor types (carpet, tiles, concrete) the robot is on using sensor data such as acceleration and velocity. Succeed and you'll help improve the navigation of robots without assistance across many different surfaces, so they won’t fall down on the job.





> The input data covers 10 sensor channels and 128 measurements per time series.
> The orientation channels encode the current angles how the robot is oriented as a quaternion. Angular velocity describes the angle and speed of motion, and linear acceleration components describe how the speed is changing at different times. 

Data has hierarchical ID system:
group_id -> series_id -> measurement_number

Each Group consists of several Series, while each Serie conrains 128 Measurements.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**1. Data Exploration**

In [ ]:
X_test = pd.read_csv("../input/X_test.csv")
X_train = pd.read_csv("../input/X_train.csv")
y_train = pd.read_csv("../input/y_train.csv")

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
y_train.head(10)

In [ ]:
y_train.shape

In [ ]:
y_train.group_id.nunique()

In [ ]:
y_train.groupby(['group_id', 'surface']).nunique().head(10)

So we have 73 groups in train data, each group has different number of series.

In [ ]:
df_temp = y_train.groupby(['surface']).count().reset_index()
plt.barh(df_temp.surface, df_temp.series_id)
plt.title("Number of 'series_id' in train data")
plt.show()

In [ ]:
surface_list = y_train.surface.unique()

*Examples of data with particular series_id*

Example 1 : series_id = 70 (Concrete)

In [ ]:
X_70 = X_train[X_train.series_id==70]

plt.figure(figsize=(30, 15))
for i, col in enumerate(X_70.columns[3:]):
    plt.subplot(3, 4, i + 1)
    plt.plot(X_70[col])
    plt.title(col)

Example 2: series_id = 12 (Carpet)

In [ ]:
X_12 = X_test[X_test.series_id==12]

plt.figure(figsize=(30, 15))
for i, col in enumerate(X_12.columns[3:]):
    plt.subplot(3, 4, i + 1)
    plt.plot(X_12[col])
    plt.title(col)

Well, some differences are obvious, especially for orientation data. In case of carpet grafs are smoother, which makes sence:) 

In [ ]:
X_train = pd.merge(y_train[['series_id', 'group_id', 'surface']], X_train, on='series_id')
X_train.head()

Histograms and density curves:

In [ ]:
sns.set_context("notebook", font_scale=1.7)
plt.figure(figsize=(30, 45))
for i, col in enumerate(X_train.columns[5:]):
    plt.subplot(5, 2, i+1)
    for surface in surface_list:
        sns.distplot(X_train[X_train.surface==surface][col], label=surface)
    plt.legend(fontsize='x-small')

Some things we can see at a glance:
- distribution of orientation data for hard tiles differs a lot
- distribution of acceleration data is more or less simmetric with the same "center" for all types of surfaces
- acceleration data for soft tiles has the lowest dispertion, for tiled - probably the highest dispertion

**2. Looking for chains in data**

*2.1 Train data*

In [ ]:
train_X = pd.read_csv('../input/X_train.csv').iloc[:,3:].values.reshape(-1,128,10)
test_X = pd.read_csv('../input/X_test.csv').iloc[:,3:].values.reshape(-1,128,10)
print('train_X shape:', train_X.shape, ', test_X shape:', test_X.shape)

In [ ]:
train_group = y_train['group_id'].values

An example of data (orientation_x, orientation_y, orientation_z and orientation_w) corresponding to one group_id (group_id=17).
Looks like one long measurement was split into small parts and then shuffled. 

In [ ]:
sns.set_context("notebook", font_scale=1.)
fig, axes = plt.subplots(1,4)
fig.set_size_inches(20,3)

for i in range(4):
    axes[i].plot(train_X[train_group == 17][:,:,i].reshape(-1))
    axes[i].grid(True)

The idea is to find out the right order and reconstruct one large contionuous mesuarement. Then we can calculate features that would be more reliable and robust.

In [ ]:
def sq_dist(a,b):
    ''' the squared euclidean distance between two samples '''
    
    return np.sum((a-b)**2, axis=1)


def find_run_edges(data, edge):
    ''' examine links between samples. left/right run edges are those samples which do not have a link on that side. '''

    if edge == 'left':
        border1 = 0
        border2 = -1
    elif edge == 'right':
        border1 = -1
        border2 = 0
    else:
        return False
    
    edge_list = []
    linked_list = []
        
    for i in range(len(data)):
        dist_list = sq_dist(data[i, border1, :4], data[:, border2, :4]) # distances to rest of samples
        min_dist = np.min(dist_list)
        closest_i   = np.argmin(dist_list) # this is i's closest neighbor
        if closest_i == i: # this might happen and it's definitely wrong
            print('Sample', i, 'linked with itself. Next closest sample used instead.')
            closest_i = np.argsort(dist_list)[1]
        dist_list = sq_dist(data[closest_i, border2, :4], data[:, border1, :4]) # now find closest_i's closest neighbor
        rev_dist = np.min(dist_list)
        closest_rev = np.argmin(dist_list) # here it is
        if closest_rev == closest_i: # again a check
            print('Sample', i, '(back-)linked with itself. Next closest sample used instead.')
            closest_rev = np.argsort(dist_list)[1]
        if (i != closest_rev): # we found an edge
            edge_list.append(i)
        else:
            linked_list.append([i, closest_i, min_dist])
            
    return edge_list, linked_list

def find_runs(data, left_edges, right_edges):
    ''' go through the list of samples & link the closest neighbors into a single run '''
    
    data_runs = []

    for start_point in left_edges:
        i = start_point
        run_list = [i]
        while i not in right_edges:
            tmp = np.argmin(sq_dist(data[i, -1, :4], data[:, 0, :4]))
            if tmp == i: # self-linked sample
                tmp = np.argsort(sq_dist(data[i, -1, :4], data[:, 0, :4]))[1]
            i = tmp
            run_list.append(i)
        data_runs.append(np.array(run_list))
    
    return data_runs

In [ ]:
train_left_edges, train_left_linked  = find_run_edges(train_X, edge='left')
train_right_edges, train_right_linked = find_run_edges(train_X, edge='right')
print('Found', len(train_left_edges), 'left edges and', len(train_right_edges), 'right edges.')

In [ ]:
train_runs = find_runs(train_X, train_left_edges, train_right_edges)

In [ ]:
y_train['run_id'] = 0
y_train['run_pos'] = 0

for run_id in range(len(train_runs)):
    for run_pos in range(len(train_runs[run_id])):
        series_id = train_runs[run_id][run_pos]
        y_train.at[ series_id, 'run_id'  ] = run_id
        y_train.at[ series_id, 'run_pos' ] = run_pos

y_train.to_csv('y_train_with_runs.csv', index=False)
y_train.tail()

In [ ]:
flat_list = [series_id for run in train_runs for series_id in run]
print(len(flat_list), len(np.unique(flat_list)))

In [ ]:
surface_cnt_train = []

for i in range(len(train_runs)):
    surface_cnt_train.append(len(np.unique(y_train['surface'].iloc[train_runs[i]])))

In [ ]:
print('Id of runs with different surfaces:')
for i in range(len(surface_cnt_train)):
    if surface_cnt_train[i] > 1:
        print(str(i) + ', number of surfaces: ' + str(surface_cnt_train[i]))

In [ ]:
fig, axes = plt.subplots(10,1, sharex=True)
fig.set_size_inches(20,15)
fig.subplots_adjust(hspace=0)

for i in range(10):
    axes[i].plot(train_X[train_runs[59]][:,:,i].reshape(-1))
    axes[i].grid(True)

In [ ]:
X_train.head()

In [ ]:
X_train = pd.merge(y_train[['series_id', 'run_id']], X_train, on='series_id')
X_train.head()

*2.2 Test data*

In [ ]:
test_X  = pd.read_csv('../input/X_test.csv' ).iloc[:,3:].values.reshape(-1,128,10)

In [ ]:
test_left_edges, test_left_linked  = find_run_edges(test_X, edge='left')
test_right_edges, test_right_linked = find_run_edges(test_X, edge='right')
print('Found', len(test_left_edges), 'left edges and', len(test_right_edges), 'right edges.')

In [ ]:
test_runs = find_runs(test_X, test_left_edges, test_right_edges)

In [ ]:
flat_list = [series_id for run in test_runs for series_id in run]
print(len(flat_list), len(np.unique(flat_list)))

In [ ]:
lost_samples = np.array([ i for i in range(len(test_X)) if i not in np.concatenate(test_runs) ])
print(lost_samples)
print(len(lost_samples))

In [ ]:
find_run_edges(test_X[lost_samples], edge='left')[1][0]

In [ ]:
lost_run = np.array(lost_samples[find_runs(test_X[lost_samples], [0], [5])[0]])
test_runs.append(lost_run)

In [ ]:
len(test_runs)

Here I tried to find test_runs, that have mesurements for 2 or more surfaces. As we can see, there are some of them (~10 runs).
Dividing these test_runs into separate runs could improve the final score.
Unfortunately I didn't use this observation because the deadline of compentition was too close. 

In [ ]:
for j in [0, 4, 5, 9, 16, 45, 47, 48, 51, 75]:   #for j in range(len(test_runs)):
    fig, axes = plt.subplots(10,1, sharex=True)
    fig.set_size_inches(20,10)
    fig.subplots_adjust(hspace=0)
    fig.suptitle('test_run: ' + str(j))
    
    for i in range(10):
        axes[i].plot(test_X[test_runs[j]][:,:,i].reshape(-1))
        axes[i].grid(True)

## test_runs(j), which look like they have 2 or more surfaces: 0, 4(?), 5, 9(?), 16, 45, 47, 48, 51, 75

In [ ]:
X_test = pd.read_csv("../input/X_test.csv")

In [ ]:
y_test = pd.DataFrame(columns = ['series_id', 'run_id', 'run_pos'])
y_test['series_id'] = X_test.series_id.unique()

In [ ]:
y_test['run_id'] = 0
y_test['run_pos'] = 0

for run_id in range(len(test_runs)):
    for run_pos in range(len(test_runs[run_id])):
        series_id = test_runs[run_id][run_pos]
        y_test.at[ series_id, 'run_id'  ] = run_id
        y_test.at[ series_id, 'run_pos' ] = run_pos

y_test.head()

In [ ]:
X_test = pd.merge(y_test[['series_id', 'run_id']], X_test, on='series_id')
X_test.head()

**3. Feature engineering**

First we translate the quaternions into euler coordinates.

In [ ]:
#thanks to https://www.kaggle.com/jesucristo/1-smart-robots-complete-notebook-0-73
import math
    
def quaternion_to_euler(x, y, z, w):
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.degrees(math.atan2(t0, t1))

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.degrees(math.asin(t2))

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.degrees(math.atan2(t3, t4))

    return X, Y, Z

In [ ]:
def fe_step0 (data):
    
    # https://www.mathworks.com/help/aeroblks/quaternionnorm.html
    # https://www.mathworks.com/help/aeroblks/quaternionmodulus.html
    # https://www.mathworks.com/help/aeroblks/quaternionnormalize.html
        
    data['norm_quat'] = data['orientation_X']**2 + data['orientation_Y']**2 + data['orientation_Z']**2 + data['orientation_W']**2 
    data['mod_quat'] = (data['norm_quat'])**0.5
    data['norm_X'] = data['orientation_X'] / data['mod_quat']
    data['norm_Y'] = data['orientation_Y'] / data['mod_quat']
    data['norm_Z'] = data['orientation_Z'] / data['mod_quat']
    data['norm_W'] = data['orientation_W'] / data['mod_quat']
    
    return data

In [ ]:
X_train = fe_step0(X_train)
X_test = fe_step0(X_test)
print(X_train.shape)
X_train.head(5)

In [ ]:
def fe_step1 (data):
    """
    Quaternions to Euler Angles
    Adding columns 'euler_x', 'euler_y', 'euler_z'
    """
    
    x, y, z, w = data['norm_X'].tolist(), data['norm_Y'].tolist(), data['norm_Z'].tolist(), data['norm_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    data['euler_x'] = nx
    data['euler_y'] = ny
    data['euler_z'] = nz
    
    return data

In [ ]:
X_train = fe_step1(X_train)
X_test = fe_step1(X_test)

print (X_train.shape)
X_train.head(5)

In [ ]:
feature_list = list(X_train.columns[6:])

In [ ]:
def feat_eng(data, test_data=False):
    """
    Feature engineering, step 2: 
    Grouping data, then calculating different statistical features.
    
    train data: is grouped by 'run_id' and 'surface'
    test data: grouped only by 'run_id', since we don't know type of surface here
    """
    
    df = pd.DataFrame()
    
    data['totl_anglr_vel'] = (data['angular_velocity_X']**2 + data['angular_velocity_Y']**2 + data['angular_velocity_Z']**2)** 0.5
    data['totl_linr_acc'] = (data['linear_acceleration_X']**2 + data['linear_acceleration_Y']**2 + data['linear_acceleration_Z']**2)**0.5
    data['acc_vs_vel'] = data['totl_linr_acc'] / data['totl_anglr_vel']
    
    def mean_change_of_abs_change(x):
        return np.mean(np.diff(np.abs(np.diff(x))))
    
    if test_data:
        col_for_group_by = ['run_id']
    else:
        col_for_group_by = ['run_id', 'surface']
    
    for col in data.columns:
        if col in ['row_id','series_id','measurement_number', 'group_id', 'run_id', 'run_pos', 'surface']:
            continue
        df[col + '_mean'] = data.groupby(col_for_group_by)[col].mean()
        df[col + '_median'] = data.groupby(col_for_group_by)[col].median()
        df[col + '_max'] = data.groupby(col_for_group_by)[col].max()
        df[col + '_min'] = data.groupby(col_for_group_by)[col].min()
        df[col + '_std'] = data.groupby(col_for_group_by)[col].std()
        df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        df[col + '_mean_abs_chg'] = data.groupby(col_for_group_by)[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df[col + '_mean_change_of_abs_change'] = data.groupby(col_for_group_by)[col].apply(mean_change_of_abs_change)
        df[col + '_abs_max'] = data.groupby(col_for_group_by)[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby(col_for_group_by)[col].apply(lambda x: np.min(np.abs(x)))
        df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2
    return df

In [ ]:
X_train = feat_eng(X_train)
X_test = feat_eng(X_test, test_data=True)

print ("New features: ",X_train.shape)

In [ ]:
feature_list.extend(['totl_anglr_vel', 'totl_linr_acc', 'acc_vs_vel'])

In [ ]:
X_train = pd.merge(y_train[['series_id', 'run_id', 'surface']], X_train, on=['run_id', 'surface'])
X_train.drop(['run_id', 'surface'], axis=1, inplace=True)
X_train.head()

In [ ]:
X_test = pd.merge(y_test[['series_id', 'run_id']], X_test, on='run_id')
X_test.drop(['run_id'], axis=1, inplace=True)
X_test.head()

**4. Train-test-split**

Here train-test-split is used for model and hyperparameters selection. For the final training and submission we use the whole train dataset.

Train-test-split:
- stratify  by surface type
- groups are not divided into test and train (all mesurements of one group go either to train or to test set) 

There is only one group_id corresponding to the surface 'Hard tiles'. So it goes to the train set.


In [ ]:
y_gr = y_train[['group_id','surface','series_id']].groupby(['group_id','surface']).count().reset_index()
y_gr.rename(columns={'series_id':'cnt'}, inplace=True)
y_gr.head()

In [ ]:
y_gr_temp = y_gr[y_gr.surface!='hard_tiles']
y_gr_train, y_gr_val = train_test_split(y_gr_temp, test_size=0.15, random_state=11, stratify=y_gr_temp.surface)
y_gr_train = y_gr_train.append(y_gr[y_gr.surface=='hard_tiles'])

In [ ]:
y_gr_val

In [ ]:
y_gr_val[['group_id', 'surface']].groupby('surface').count()

In [ ]:
#y_gr_train.head(3)
y_gr_train[['group_id', 'surface']].groupby('surface').count()

In [ ]:
y_train_data = y_train.merge(y_gr_train.group_id, on='group_id')

In [ ]:
y_val_data = y_train.merge(y_gr_val.group_id, on='group_id')

In [ ]:
y_train_data[['series_id', 'surface']].groupby('surface').count()

In [ ]:
y_val_data[['group_id', 'surface']].groupby('surface').count() 

Ok, now the group numbers are good stratified.

In [ ]:
X = y_train_data.merge(X_train, on='series_id')
X = X.drop(['group_id', 'run_id', 'run_pos', 'series_id', 'surface'], axis=1)
X = np.array(X)

X_val = y_val_data.merge(X_train, on='series_id')
X_val = X_val.drop(['group_id', 'run_id', 'run_pos', 'series_id', 'surface'], axis=1)
X_val = np.array(X_val)

In [ ]:
X_test = X_test.sort_values('series_id')

X_t = X_test.drop(['series_id'], axis=1)
X_t = np.array(X_t)

In [ ]:
print(X.shape, X_val.shape, X_t.shape) 

In [ ]:
n_featurs = X.shape[1]
print(n_featurs)

**5. Training Model (Random Forest)**

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=11, verbose=1,
                                class_weight='balanced',
                                random_state=11, n_jobs=10)

In [ ]:
rf_clf.fit(X, y_train_data['surface'])

In [ ]:
print('Accuracy on train data: ', rf_clf.score(X, y_train_data['surface']))
print('Accuracy on validation data: ', rf_clf.score(X_val, y_val_data['surface']))

Confusion matrix for training and validation data:

In [ ]:
# thanks to the author of this kernel: https://www.kaggle.com/artgor/where-do-the-robots-drive
import itertools
def plot_confusion_matrix(truth, pred, classes, normalize=False, title=''):
    cm = confusion_matrix(truth, pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title, size=15)
    plt.colorbar(fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False)
    plt.tight_layout()


In [ ]:
pred_train = rf_clf.predict(X)
pred_val = rf_clf.predict(X_val) 

In [ ]:
plot_confusion_matrix(y_train_data['surface'], pred_train, classes=rf_clf.classes_, title='Confusion matrix - Train data')

In [ ]:
plot_confusion_matrix(y_val_data['surface'], pred_val, classes=rf_clf.classes_, title='Confusion matrix - Validation data')

As we can see, our model is overfitted. But the score on validation set is still quite good for this competion.

Actually, as we know now after the end of competition, the CV score is very close to the private leaderboard score, this means our CV-strategy works well.

It's interesting to take a look at feature importance:

In [ ]:
feat_imp = pd.DataFrame(columns=['importance', 'feature'])
feat_imp['importance'] = rf_clf.feature_importances_
feat_imp['feature'] =  X_train.columns[1:]
feat_imp.sort_values(by='importance', ascending=False).head(10)

8 of top-10 most important features is Mean Absolute Change. It seems to be the most usefull statistical characteristic here. 

In [ ]:
color_list = list('kkkkbbbgggccyyyymmmrrr')

fig, ax = plt.subplots(figsize=(14, 5))
plt.plot(feat_imp.importance)
plt.xticks(np.arange(0, 270, 12))
plt.grid()
for i in range(len(feature_list)):
    t = feature_list[i]
    plt.text(6+12*i, -0.004, t, ha='center', va='top', rotation=90, color=color_list[i], fontsize=12)
    
plt.title("Feature importance and groups of features")
plt.show()


As we can see here, the most important groups of features are angular velocity and linear acceleration (coordinate wise as well as their absolute values and their ratio).  
Orientation data are less usefull.

Now, when we've chosen the model, we train it for submission on the whole dataset.

In [ ]:
X = y_train.merge(X_train, on='series_id')
X = X.drop(['group_id', 'run_id', 'run_pos', 'series_id', 'surface'], axis=1)
X = np.array(X)

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=11, verbose=1,
                                class_weight='balanced',
                                random_state=11, n_jobs=10)

In [ ]:
rf_clf.fit(X, y_train['surface'])

**6. Submission**

In [ ]:
subm_result = rf_clf.predict(X_t)

In [ ]:
submission = pd.DataFrame(columns = ['series_id', 'surface'])
submission['series_id'] = X_test.series_id.unique()
submission['surface'] = subm_result

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()